In [5]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
import webbrowser

import random
import numpy as np

In [2]:
class SchellingAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.type = random.uniform(0,1)
        
    def step(self):
        print(f'Hi, I am agent {self.unique_id}')

In [3]:
class SchellingModel(Model):
    def __init__(self, N, width, height):
        self.grid = MultiGrid(width, height, True)
        self.num_agents = N
        self.schedule = RandomActivation(self)
        for i in range(0, N):
            a = SchellingAgent(i, self)
            self.schedule.add(a)
            
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            
    def step(self):
        self.schedule.step()
            

In [4]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    return portrayal

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(SchellingModel,
                       [grid],
                       "Schelling Model",
                       {"N":100, "width":10, "height":10})
server.port = 8522 # The default
server.launch(open_browser=False)

Interface starting at http://127.0.0.1:8522


RuntimeError: This event loop is already running

In [8]:
webbrowser.open('127.0.0.1:8522')

True